In [8]:
import pyspark

spark = pyspark.sql.SparkSession \
    .builder \
    .appName("COVID-CXR META ETL") \
    .config("spark.some.config.option", "xxx") \
    .getOrCreate()

try:
    # once intialized, it's global in the jupyter notebook kernal; one context per notebook
    sc = pyspark.SparkContext.getOrCreate('local[*]')
except:
    sc = pyspark.SparkContext('local[*]')

sqlc = pyspark.SQLContext

In [97]:
from src.etl_spark import *

rdd = spark_etl()
a = rdd.collect()

In [103]:
len(CXRs)

22049

In [100]:
META = pd.DataFrame()
for i, cxr in enumerate(CXRs):
    META.loc[i, 'patient'] = cxr.pid
    META.loc[i, 'img'] = cxr.fn_src
    META.loc[i, 'label'] = cxr.label
    META.loc[i, 'src'] = cxr.src

<module 'pandas' from 'C:\\Users\\conno\\anaconda3\\envs\\covid\\lib\\site-packages\\pandas\\__init__.py'>

In [42]:
sc

<SparkContext master=local[*] appName=COVID-CXR META ETL>

In [35]:
spark

In [49]:
for i in dir(rdd):
    if i[0] != '_':
        print(i)

aggregate
aggregateByKey
barrier
cache
cartesian
checkpoint
coalesce
cogroup
collect
collectAsMap
collectWithJobGroup
combineByKey
context
count
countApprox
countApproxDistinct
countByKey
countByValue
ctx
distinct
filter
first
flatMap
flatMapValues
fold
foldByKey
foreach
foreachPartition
fullOuterJoin
getCheckpointFile
getNumPartitions
getStorageLevel
glom
groupBy
groupByKey
groupWith
histogram
id
intersection
isCheckpointed
isEmpty
isLocallyCheckpointed
is_cached
is_checkpointed
join
keyBy
keys
leftOuterJoin
localCheckpoint
lookup
map
mapPartitions
mapPartitionsWithIndex
mapPartitionsWithSplit
mapValues
max
mean
meanApprox
min
name
partitionBy
partitioner
persist
pipe
randomSplit
reduce
reduceByKey
reduceByKeyLocally
repartition
repartitionAndSortWithinPartitions
rightOuterJoin
sample
sampleByKey
sampleStdev
sampleVariance
saveAsHadoopDataset
saveAsHadoopFile
saveAsNewAPIHadoopDataset
saveAsNewAPIHadoopFile
saveAsPickleFile
saveAsSequenceFile
saveAsTextFile
setName
sortBy
sortByKey
st

In [19]:
sqlContext = sc.sqlContext

AttributeError: 'SparkContext' object has no attribute 'sqlContext'

## meta

In [ ]:
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

In [ ]:
params['data']['image_size']

In [ ]:
META = etl()

In [ ]:
print('label distribution: ', Counter(META.label), sep='\n')

## train / test split

In [ ]:
META = dataset_split(META)
META.head()

In [ ]:
print('label integer map: ', params['train']['labelmap'], sep='\n', end='\n\n')
for ds in (0, 1):
    print('{0} data from covid datasets: '.format({0: 'test', 1: 'train'}[ds]), Counter(META[META.train==ds].label), sep='\n', end='\n\n')

In [ ]:
# cache image meta
if not os.path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH)
with open(os.path.join(SAVE_PATH, 'meta'), 'wb') as pickle_file:
    pickle.dump(META, pickle_file, pickle.HIGHEST_PROTOCOL)
    print(f"META data saved at {os.path.join(SAVE_PATH, 'meta')}")

In [ ]:
%%time
if False:
    # -------------------------------------------------------------------------------------------------------------------
    # imagge file to matrix  
    # NOTE: this needs model to load all data at once into cache which take up too much storage, 
    #       making computing inefficient, NOT GOOD
    # -------------------------------------------------------------------------------------------------------------------
    # train_data / test_data structure: 
    #                        {'covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer},
    #                        '!covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer}}
    TRAIN_DATA, TEST_DATA = datafromfile(META)
    # a sanity check
    print(f"train covid sample #: {len(TRAIN_DATA['covid']['label'])}")
    print(f"train !covid sample #: {len(TRAIN_DATA['!covid']['label'])}")
    print(f"test  covid sample #: {len(TEST_DATA['covid']['label'])}")
    print(f"test  !covid sample #: {len(TEST_DATA['!covid']['label'])}")
    # cache image data
    with open(os.path.join(SAVE_PATH, 'train.data'), 'wb') as pickle_file:
        pickle.dump(TRAIN_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'train.data')}")
    with open(os.path.join(SAVE_PATH, 'test.data'), 'wb') as pickle_file:
        pickle.dump(TEST_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'test.data')}")
else:
    # merge imgaes together instead, and transform image to batches of matrix during training
    mergesoure(META)

In [ ]:
# sanity check for image shape, they need to consistent with model input
import cv2
import glob
import numpy as np
for i in np.random.choice(META.index, 10):
    print(i, cv2.imread(META.loc[i, 'imgid']).shape)